In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [92]:
load = pd.read_pickle("load.pickle")  
loadfrcst = pd.read_pickle("loadfrcst.pickle")  
price = pd.read_pickle("price.pickle")  
pricefrcst = pd.read_pickle("pricefrcst.pickle")  

load['Date'] = pd.to_datetime(load['Date'], format='%m/%d/%Y %I:%M:%S %p')


In [93]:
load

,Date,Load,Zone
0,2006-02-06 00:00:00,995.6000,CAPITL
1,2006-02-06 00:00:00,1767.1000,CENTRL
2,2006-02-06 00:00:00,556.7000,DUNWOD
3,2006-02-06 00:00:00,992.3000,GENESE
4,2006-02-06 00:00:00,966.9000,HUD VL
...,...,...,...
1566418,2023-09-24 20:00:00,774.1452,MHK VL
1566419,2023-09-24 20:00:00,254.1218,MILLWD
1566420,2023-09-24 20:00:00,5097.6534,N.Y.C.
1566421,2023-09-24 20:00:00,638.6140,NORTH


# duplicate
delete duplicates according to 
1. delete 0
2. take an average of duplicates

In [4]:
duplicates = load[load.duplicated(subset=['Date', 'Zone'])]
print(duplicates.shape)
# Findings: 2 types of duplicates
type1 = load[load['Date'] == '2006-10-29 01:00:00'] # delete 0; 33 rows
type2 = load[load['Date'] == '2022-11-06 01:00:00'] # average; 3388 rows

(3421, 3)


In [5]:
# type1
print(load.shape)
load_drop = load.drop(load[(load['Date'].isin(duplicates['Date'])) & (load['Load']==0)].index) # 33 rows
print(load_drop.shape)
load_drop


(1566423, 3)
(1566390, 3)


,Date,Load,Zone
0,2006-02-06 00:00:00,995.6000,CAPITL
1,2006-02-06 00:00:00,1767.1000,CENTRL
2,2006-02-06 00:00:00,556.7000,DUNWOD
3,2006-02-06 00:00:00,992.3000,GENESE
4,2006-02-06 00:00:00,966.9000,HUD VL
...,...,...,...
1566418,2023-09-24 20:00:00,774.1452,MHK VL
1566419,2023-09-24 20:00:00,254.1218,MILLWD
1566420,2023-09-24 20:00:00,5097.6534,N.Y.C.
1566421,2023-09-24 20:00:00,638.6140,NORTH


In [6]:
# type2
duplicates_2 = load_drop[load_drop.duplicated(subset=['Date', 'Zone'])]
print(duplicates_2.shape)
load_drop = load_drop.groupby(['Date', 'Zone'])['Load'].mean().reset_index()
print(load_drop.shape)
load_drop

(3388, 3)
(1563002, 3)


,Date,Zone,Load
0,2006-02-06 00:00:00,CAPITL,995.6000
1,2006-02-06 00:00:00,CENTRL,1767.1000
2,2006-02-06 00:00:00,DUNWOD,556.7000
3,2006-02-06 00:00:00,GENESE,992.3000
4,2006-02-06 00:00:00,HUD VL,966.9000
...,...,...,...
1562997,2023-09-24 20:00:00,MHK VL,774.1452
1562998,2023-09-24 20:00:00,MILLWD,254.1218
1562999,2023-09-24 20:00:00,N.Y.C.,5097.6534
1563000,2023-09-24 20:00:00,NORTH,638.6140


In [7]:
load = load_drop

# imputation

In [9]:
load['Date'].min()

Timestamp('2006-02-06 00:00:00')

In [10]:
# 所有的 hour 時間點
hours_df = pd.DataFrame({'Date': pd.date_range(start = load['Date'].min(), 
                                               end = load['Date'].max(), 
                                               freq = '1H') })
print(hours_df.shape)
hours_df

(154557, 1)


,Date
0,2006-02-06 00:00:00
1,2006-02-06 01:00:00
2,2006-02-06 02:00:00
3,2006-02-06 03:00:00
4,2006-02-06 04:00:00
...,...
154552,2023-09-24 16:00:00
154553,2023-09-24 17:00:00
154554,2023-09-24 18:00:00
154555,2023-09-24 19:00:00


In [44]:
# check 1 zone for loops
zone = 'CAPITL'
load_zone = load.loc[load['Zone'] == zone,]  # 142404
print(load_zone.shape)
result = pd.merge(hours_df, load_zone, on='Date', how='left')
result['Zone'] = zone
print(result.shape)
print(result['Load'].isna().sum())
print(result[result['Load'].isna()])
result['Load'].interpolate(method='linear', inplace=True)
print(result['Load'].isna().sum())
print(result.iloc[35:40,])

(142093, 3)
(154557, 3)
12464
                      Date    Zone  Load
37     2006-02-07 13:00:00  CAPITL   NaN
38     2006-02-07 14:00:00  CAPITL   NaN
39     2006-02-07 15:00:00  CAPITL   NaN
40     2006-02-07 16:00:00  CAPITL   NaN
41     2006-02-07 17:00:00  CAPITL   NaN
...                    ...     ...   ...
151157 2023-05-06 05:00:00  CAPITL   NaN
151918 2023-06-06 22:00:00  CAPITL   NaN
151919 2023-06-06 23:00:00  CAPITL   NaN
153724 2023-08-21 04:00:00  CAPITL   NaN
153726 2023-08-21 06:00:00  CAPITL   NaN

[12464 rows x 3 columns]
0
                  Date    Zone         Load
35 2006-02-07 11:00:00  CAPITL  1429.100000
36 2006-02-07 12:00:00  CAPITL  1417.400000
37 2006-02-07 13:00:00  CAPITL  1415.358889
38 2006-02-07 14:00:00  CAPITL  1413.317778
39 2006-02-07 15:00:00  CAPITL  1411.276667


In [51]:
zones = load['Zone'].unique()

load_impute = pd.DataFrame()
for zone in zones:

    # one zone in one loop
    load_zone = load.loc[load['Zone'] == zone,]

    # merge to hourly hours
    result = pd.merge(hours_df, load_zone, on='Date', how='left')
    result['Zone'] = zone

    # imputation
    result['Load'].interpolate(method='linear', inplace=True)

    load_impute = pd.concat([load_impute, 
                             result], 
                            axis=0)
    


In [98]:
print(hours_df.shape[0]*11)
print(load_impute.shape)
zones

1700127
(1700127, 3)


array(['CAPITL', 'CENTRL', 'DUNWOD', 'GENESE', 'HUD VL', 'LONGIL',
       'MHK VL', 'MILLWD', 'N.Y.C.', 'NORTH', 'WEST'], dtype=object)

# pd to np array

In [57]:
load = load_impute
# 使用 pd.date_range 生成日期序列 '2006-02-06' to '2023-09-17'
days = pd.date_range(start = load['Date'].min(), 
                     end = load['Date'].max() - pd.Timedelta(days=7), 
                     freq = '1D') 


In [72]:
load_array = np.empty((1, 11, 168))
load_array

array([[[6.92044275e-310, 6.92044275e-310, 4.64262870e-310, ...,
         1.45769922e+003, 6.92035543e-310, 1.42909961e+003],
        [1.41740000e+003, 8.48798317e-314, 3.60739285e-313, ...,
         2.28979883e+003, 4.64262870e-310, 2.32459961e+003],
        [4.64262870e-310, 2.27312500e+003, 4.64262870e-310, ...,
         7.90199707e+002, 0.00000000e+000, 7.86000000e+002],
        ...,
        [2.92800000e+002, 2.92721667e+002, 2.92643333e+002, ...,
         6.48850000e+003, 6.58510000e+003, 6.61960000e+003],
        [6.60340000e+003, 6.59379333e+003, 6.58418667e+003, ...,
         8.16700000e+002, 8.04900000e+002, 8.02400000e+002],
        [7.91000000e+002, 7.90738889e+002, 7.90477778e+002, ...,
         2.12900000e+003, 2.13630000e+003, 2.13220000e+003]]])

In [77]:
dim = (1, 11, 168)
load_array = np.empty(dim)
for day in days:
    # 挑選 7 天資料
    start_date = pd.to_datetime(day)
    
    end_date = start_date + pd.Timedelta(days=7)
    
    selected_data = load.loc[(load['Date'] >= start_date) & (load['Date'] < end_date),:]
    
    # df -> numpy array [obs(number of loops) = 6433, channel(number of zones) = 11, length(number of hours) = 168]
    pivot_df = selected_data.pivot(index='Zone', columns='Date', values='Load') 
    pivot_np = pivot_df.to_numpy() # (zone, Date)

    pivot_np = pivot_np.reshape(dim) # (1,11,168)
    load_array = np.concatenate((load_array, pivot_np), axis=0)

load_array = load_array[1:] # delete the 1 st slice created by np.empty(dim)

In [104]:
print(days.shape)
print(load_array.shape)

(6433,)
(6433, 11, 168)


In [105]:
np.save('load_array.npy', load_array)

In [101]:
# check the 1st dimension
print(load['Load'][(load['Date'] == '2006-02-06 00:00:00') & (load['Zone'] == 'CAPITL')]) # 995.6000
print(load_array[0,0,0])
print(load['Load'][(load['Date'] == '2006-02-07 00:00:00') & (load['Zone'] == 'CAPITL')])
print(load_array[1,0,0])		
print(load['Load'][(load['Date'] == '2023-09-17 00:00:00') & (load['Zone'] == 'CAPITL')])
print(load_array[-1,0,0])	

0    995.6
Name: Load, dtype: float64
995.6
264    1067.3
Name: Load, dtype: float64
1067.3
1564344    1036.491
Name: Load, dtype: float64
1036.491


In [103]:
# check the 2nd dimension
print(load['Load'][(load['Date'] == '2006-02-06 00:00:00') & (load['Zone'] == 'CAPITL')]) # 995.6000
print(load_array[0,0,0])
print(load['Load'][(load['Date'] == '2006-02-06 00:00:00') & (load['Zone'] == 'CENTRL')])
print(load_array[0,1,0])	
print(load['Load'][(load['Date'] == '2006-02-06 00:00:00') & (load['Zone'] == 'WEST')])
print(load_array[0,10,0])	


print(load['Load'][(load['Date'] == '2023-09-17 00:00:00') & (load['Zone'] == 'CAPITL')]) # 995.6000
print(load_array[-1,0,0])
print(load['Load'][(load['Date'] == '2023-09-17 00:00:00') & (load['Zone'] == 'CENTRL')])
print(load_array[-1,1,0])	
print(load['Load'][(load['Date'] == '2023-09-17 00:00:00') & (load['Zone'] == 'WEST')])
print(load_array[-1,10,0])	

0    995.6
Name: Load, dtype: float64
995.6
1    1767.1
Name: Load, dtype: float64
1767.1
10    1660.0
Name: Load, dtype: float64
1660.0
1564344    1036.491
Name: Load, dtype: float64
1036.491
1564345    1363.9602
Name: Load, dtype: float64
1363.9602
1564354    1376.1895
Name: Load, dtype: float64
1376.1895
